In [ ]:
from dataManager import getData

In [ ]:
target_data = getData('M1800')

In [ ]:
train_index = int(target_data.shape[0]*0.7)

In [ ]:
training_data = target_data.iloc[:train_index,:]['ret']
testing_data = target_data.iloc[train_index:,:]['ret']

# 選擇落後期

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(20, 20))

ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

plot_acf(training_data, ax=ax) 
plot_pacf(training_data, ax=ax2) 

plt.show()

# ARMA

In [ ]:
from statsmodels.tsa.arima_model import ARMA


arma_model = ARMA(training_data, (1, 1)).fit()

# 模型訓練結果

In [ ]:
arma_model.summary()

In [ ]:
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

ax.set_title('training predict')
ax.plot(arma_model.predict())

ax2.set_title('Real')
ax2.plot(training_data, c='k')

plt.show()

# 模型樣本外預測

In [ ]:
arma_model.forecast?

In [ ]:
forecast_step = 30
predict_ret, predict_std, predict_interval = arma_model.forecast(forecast_step)

In [ ]:
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

ax.set_title('Predict')
ax.plot(predict_ret)

ax2.set_title('Real')
ax2.plot(testing_data.iloc[:forecast_step], c='k')

plt.show()

# 重新調整落後期

In [ ]:
%%time
from statsmodels.tsa.stattools import arma_order_select_ic


arma_order_select_ic_res = arma_order_select_ic(
    training_data, 
    max_ar=5,
    max_ma=5, 
    ic=['aic'], 
    trend='c'
)

In [ ]:
arma_order_select_ic_res

In [ ]:

better_arma_model = ARMA(training_data, (4, 4)).fit()

In [ ]:
better_arma_model.summary()

In [ ]:
forecast_step = 30
predict_ret, predict_std, predict_interval = better_arma_model.forecast(forecast_step)

fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

ax.set_title('Predict')
ax.plot(predict_ret)

ax2.set_title('Real')
ax2.plot(testing_data.iloc[:forecast_step], c='k')

plt.show()